<a href="https://colab.research.google.com/github/Pi-vardhan/Pi-vardhan/blob/Project/Water_Tank_Draining_Prediction_AIML_FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mounting Drive**

In [253]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Libraries

In [254]:
import numpy 
import math 
import random
import pandas 

**Data Input**

In [255]:
tank = pandas.read_csv('/content/drive/MyDrive/Tank.csv')
tank.head(7)

,DOW,Filled Time,Drained Time
0,Sunday,18,8
1,Monday,10,11
2,Tuesday,13,7
3,Wednesday,8,16
4,Thursday,18,16
5,Friday,18,22
6,Saturday,24,14


In [256]:
print(tank.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DOW           303 non-null    object
 1   Filled Time   303 non-null    int64 
 2   Drained Time  303 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 7.2+ KB
None


**Calculate Difference** [ Time Taken to Drain = Drain Time - Filled Time ]

In [257]:
ftime=tank['Filled Time']
dtime=tank['Drained Time']
timetook=dtime-ftime
tank = pandas.concat([tank, timetook],axis = 1)
#tank.drop(['Filled Time'],axis=1,inplace = True)
tank.drop(['Drained Time'],axis=1,inplace = True)
tank = tank.rename(columns=({0:'Drain Time'}))
#tank.head(7)

No Negative Numbers

In [258]:
data = tank['Drain Time']
i=0
for j in data:
  if j<1:
    tank.loc[i,'Drain Time'] = (j*-1)+24
  i=i+1
tank.head(7)

,DOW,Filled Time,Drain Time
0,Sunday,18,34
1,Monday,10,1
2,Tuesday,13,30
3,Wednesday,8,8
4,Thursday,18,26
5,Friday,18,4
6,Saturday,24,34


Outliers Detection

In [259]:
def detect_outliers_zscore(data):
    outliers = []
    thres = 1.45
    mean = numpy.mean(data)
    std = numpy.std(data)
    for i in data:
        z_score = (i-mean)/std
        if (numpy.abs(z_score) > thres):
            outliers.append(i)
    return outliers

In [260]:
sample_outliers_Drain_Time = detect_outliers_zscore(tank['Drain Time'])
#sample_outliers_Fllled = detect_outliers_zscore(tank['Filled Time'])
#print('No.of Outliers in "Fllled" Column:', len(sample_outliers_Fllled))
#print("Outliers in 'Filled Time' column : ", sample_outliers_Fllled)
print('No.of Outliers in "Drain Time" Column:', len(sample_outliers_Drain_Time))
print("Outliers in 'Drain Time' column : ", sample_outliers_Drain_Time)

No.of Outliers in "Drain Time" Column: 26
Outliers in 'Drain Time' column :  [1, 44, 45, 46, 1, 1, 1, 1, 46, 47, 1, 1, 1, 44, 44, 1, 44, 45, 1, 1, 1, 1, 63, 47, 75, 106]


In [261]:
from scipy.stats import zscore
z = numpy.abs(zscore(tank['Drain Time']))
tank['Zscore_Drain_Time'] = z
outliers_Drain_Time=len(tank[tank['Zscore_Drain_Time']>1])
data_z = tank[tank['Zscore_Drain_Time']<1.45]
tank=data_z.drop(['Zscore_Drain_Time'], axis=1)

In [262]:
print(tank.describe())

       Filled Time  Drain Time
count   277.000000  277.000000
mean     13.866426   22.162455
std       6.949105   12.422468
min       1.000000    2.000000
25%       7.000000    9.000000
50%      15.000000   26.000000
75%      20.000000   32.000000
max      26.000000   43.000000


Missing Value Treatment

In [263]:
print(tank.isnull().sum())

DOW            0
Filled Time    0
Drain Time     0
dtype: int64


Correlation Matrix


Catergorical CONVERSION

In [264]:
status = pandas.get_dummies(tank['DOW'], drop_first = True)
tank = pandas.concat([tank, status],axis = 1)
tank.drop(['DOW'],axis = 1, inplace = True)
tank

,Filled Time,Drain Time,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,18,34,0,0,1,0,0,0
2,13,30,0,0,0,0,1,0
3,8,8,0,0,0,0,0,1
4,18,26,0,0,0,1,0,0
5,18,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
296,8,5,1,0,0,0,0,0
299,11,6,0,0,0,1,0,0
300,18,29,0,0,0,0,0,0
301,14,36,0,1,0,0,0,0


DATA Split

In [265]:
from sklearn.model_selection import train_test_split
tank_train, tank_test= train_test_split(tank,train_size = 0.80, test_size = 0.20, random_state = 1438)

DATA Normalization

In [266]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
nv = ['Drain Time','Filled Time']
tank_train[nv] = scaler.fit_transform(tank_train[nv])
tank_test[nv]= scaler.fit_transform(tank_test[nv])

tank_train

,Filled Time,Drain Time,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
51,0.68,0.780488,0,0,0,1,0,0
176,0.40,0.097561,0,0,1,0,0,0
262,0.08,0.146341,0,0,0,0,1,0
88,0.72,0.073171,0,0,0,0,1,0
187,0.80,0.024390,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
191,0.76,0.000000,1,0,0,0,0,0
228,0.64,0.707317,0,0,0,0,0,1
233,0.32,0.658537,1,0,0,0,0,0
175,0.84,0.853659,0,1,0,0,0,0


INPUT - OUTPUT Data Split

In [267]:
x_train = tank_train
x_test = tank_test
y_train = tank_train.pop('Drain Time')
y_test = tank_test.pop('Drain Time')

**Model Training**

In [268]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression()

In [269]:
m = regressor.coef_
c = regressor.intercept_,
print("Estimated model weights, m:" , m)
print("Estimated model intercept, c:" , c)

Estimated model weights, m: [ 0.62524158 -0.03439071  0.00214514 -0.04980479 -0.02486836  0.0609134
  0.01899781]
Estimated model intercept, c: (0.15329963019637866,)


Prediction on Training & Testing Data

In [270]:
y_pred_test =  regressor.predict(x_test)
y_pred_train =  regressor.predict(x_train)

Error Metric Computation

In [271]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error :', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error      :', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error    :', mae)

Root mean square error : 0.2528456294331319
Mean square error      : 0.06393091232343666
Mean absolute error    : 0.2078662608424524


In [272]:
#Testing Accuracies
rmse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error :', rmse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error      :', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error    :', mae)
     

Root mean square error : 0.24187018649233114
Mean square error      : 0.05850118711383504
Mean absolute error    : 0.20665339282979112


SAVE Model

In [273]:
import pickle
# Save the model
filename = 'model.pkl'
pickle.dump(regressor, open(filename, 'wb'))

Deployment

In [274]:
list_of_columns = tank.columns
input_data = pandas.DataFrame(columns=list_of_columns)
input_data.drop(['Drain Time'],axis='columns',inplace=True)
loop=m=t=w=th=f=s=su=0

while loop==0:
  day = str(input('Day:'))
  if day == 'Monday':
    m=1
    break
  elif day == 'Tuesday':
    t=1
    break
  elif day == 'Wednesday':
    w=1
    break
  elif day == 'Thursday':
    th=1
    break
  elif day == 'Saturday':
    s=1
    break
  elif day == 'Sunday':
    su=1
    break
  elif day == 'Friday':
    break
  else:
    print('Invalid Day!!')

input_data.at[0,'Monday'] = m
input_data.at[0,'Tuesday'] = t
input_data.at[0,'Wednesday'] = w
input_data.at[0,'Thursday'] = th
input_data.at[0,'Saturday'] = s
input_data.at[0,'Sunday'] = su

input_data.at[0,'Filled Time'] = time = int(input('Enter Time: '))
while time>23 or time<0:
  print('Invalid Time!!')
  input_data.at[0,'Filled Time'] = time = int(input('Enter Time: '))

input_data['Filled Time']=(input_data['Filled Time']-tank['Filled Time'].min())/(tank['Filled Time'].max()-tank['Filled Time'].min())
input_data['Monday']=(input_data['Monday']-tank['Monday'].min())/(tank['Monday'].max()-tank['Monday'].min())
input_data['Tuesday']=(input_data['Tuesday']-tank['Tuesday'].min())/(tank['Tuesday'].max()-tank['Tuesday'].min())
input_data['Wednesday']=(input_data['Wednesday']-tank['Wednesday'].min())/(tank['Wednesday'].max()-tank['Wednesday'].min())
input_data['Thursday']=(input_data['Thursday']-tank['Thursday'].min())/(tank['Thursday'].max()-tank['Thursday'].min())
input_data['Saturday']=(input_data['Saturday']-tank['Saturday'].min())/(tank['Saturday'].max()-tank['Saturday'].min())
input_data['Sunday']=(input_data['Sunday']-tank['Sunday'].min())/(tank['Sunday'].max()-tank['Sunday'].min())

Day:#$%
Invalid Day!!
Day:Sunday
Enter Time: 235
Invalid Time!!
Enter Time: 23


Defining Basic Codes

In [275]:
def ampm(x):
  if x<12 :
    return 'AM'
  else:
    return 'PM'

def t12(x):
  if x>12:
    return x-12
  else:
    return x

def frint(x):
  if x>47:
    x = x-48
    print('Water Tank will be Drained by', t12(x),ampm(x),'Day After Tommorow.')
  elif x>23:
    x = x-24
    print('Water Tank Will Be Emptied by', t12(x),ampm(x), 'Tommorow.') 
  else:
    print('Water Tank Will Be Emptied by',t12(x),ampm(x),'Today.')

# ***Final Outputs***

In [276]:
y_pred =  regressor.predict(input_data)
Dt = y_pred*(tank['Drain Time'].max()-tank['Drain Time'].min())+tank['Drain Time'].min()
print('Predicted Drain Time =',int(Dt),'hrs')
et = time+int(Dt)
frint(et)

Predicted Drain Time = 28 hrs
Water Tank will be Drained by 3 AM Day After Tommorow.
